In [18]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import plotly_express as exp
from datetime import datetime

In [2]:
df = pd.read_csv('./data/total.csv', sep=';', decimal = ',')

/home/alvaro/d-projects/venv/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
df.head(3)

,codigo,status,situacao_debito,codigo_boleto,valor_debito,data_debito,operacao_origem,baixa_boleto,data_vencimento,data_pagamento,codigo_atendente,nome_atendente,quem_debitou,debitos_cliente,dias_corridos,valor_principal
0,11,X,QUITADO,88,1.0,07/05/2014,VENDAS,00000011,20/05/2014,16/05/2014,5.0,NELCI,NELCI RODRIGUES FERREIRA,/ /,0,0.0
1,11,X,QUITADO,281,23.2,16/05/2014,VENDAS,00000011,20/05/2014,16/05/2014,1.0,MARCELO ANDRIOLI,NELCI RODRIGUES FERREIRA,/ /,0,0.0
2,11,X,QUITADO,282,274.0,16/05/2014,VENDAS,00000012,20/05/2014,16/05/2014,1.0,MARCELO ANDRIOLI,NELCI RODRIGUES FERREIRA,/ /,0,0.0


In [4]:
df.dtypes

codigo               object
status               object
situacao_debito      object
codigo_boleto         int64
valor_debito        float64
data_debito          object
operacao_origem      object
baixa_boleto         object
data_vencimento      object
data_pagamento       object
codigo_atendente    float64
nome_atendente       object
quem_debitou         object
debitos_cliente      object
dias_corridos         int64
valor_principal      object
dtype: object

In [5]:
del df['status']
del df['codigo_boleto']
del df['baixa_boleto']
del df['debitos_cliente']
del df['valor_principal']
del df['nome_atendente']
del df['codigo_atendente']

In [6]:
df.columns

Index(['codigo', 'situacao_debito', 'valor_debito', 'data_debito',
       'operacao_origem', 'data_vencimento', 'data_pagamento', 'quem_debitou',
       'dias_corridos'],
      dtype='object')

In [7]:
df['operacao_origem'].unique()

array(['VENDAS', 'RESIDUO'], dtype=object)

In [8]:
df['situacao_debito'].unique()

array(['QUITADO', 'DEVEDOR'], dtype=object)

In [9]:
df.groupby('situacao_debito').agg({'valor_debito': 'sum'})

,valor_debito
situacao_debito,
DEVEDOR,29364.91
QUITADO,1047444.69


In [13]:
df.dtypes

codigo              object
situacao_debito     object
valor_debito       float64
data_debito         object
operacao_origem     object
data_vencimento     object
data_pagamento      object
quem_debitou        object
dias_corridos        int64
dtype: object

In [34]:
len(df['data_pagamento'].unique())

1553

In [40]:
df_quitado = df[df['situacao_debito'] == 'QUITADO']

In [41]:
df_quitado['data_pagamento'] = df_quitado['data_pagamento'].map(lambda x: datetime.strptime(x, '%d/%m/%Y'))

<ipython-input-41-f8411f9f7d77>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_quitado['data_pagamento'] = df_quitado['data_pagamento'].map(lambda x: datetime.strptime(x, '%d/%m/%Y'))


In [49]:
df_quitado['week_day'] = df_quitado['data_pagamento'].map(lambda x: x.weekday())
df_quitado['month'] = df_quitado['data_pagamento'].map(lambda x: x.month)
df_quitado['year'] = df_quitado['data_pagamento'].map(lambda x: x.year)

<ipython-input-49-b79b45631852>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_quitado['week_day'] = df_quitado['data_pagamento'].map(lambda x: x.weekday())
<ipython-input-49-b79b45631852>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_quitado['month'] = df_quitado['data_pagamento'].map(lambda x: x.month)
<ipython-input-49-b79b45631852>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cav

In [46]:
df_quitado.groupby('week_day').sum()

,valor_debito,dias_corridos
week_day,,
0,190383.64,4772
1,166261.92,8521
2,176355.47,5275
3,194458.08,11614
4,200917.02,11004
5,118362.00,4480
6,706.56,200


In [80]:
df_year = df_quitado.groupby('year')['valor_debito'].sum()
df_year = df_year[df_year.index != 2009]

In [91]:
exp.bar(x=df_year.index, y=df_year.values, )

In [167]:
df_quitado['month_year'] = df_quitado['month'].astype(str) + '/' + df_quitado['year'].astype(str)
df_quitado['month_year_s'] = [x.split('/')[1]+x.split('/')[0].rjust(2, '0') for x in df_quitado['month_year']]
df_quitado = df_quitado.sort_values('month_year_s')

<ipython-input-167-73733cf32519>:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-167-73733cf32519>:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [168]:
df_month_year = df_quitado.groupby('month_year')['valor_debito'].sum()
df_month_year = df_month_year[df_month_year.index != '1/2009']

In [169]:
exp.bar(x=df_month_year.index, y=df_month_year.values)

In [170]:
df_quitado.head(20)

,codigo,situacao_debito,valor_debito,data_debito,operacao_origem,data_vencimento,data_pagamento,quem_debitou,dias_corridos,week_day,month,year,month_year,month_year_s
9744,37,QUITADO,56.51,07/01/2020,RESIDUO,07/01/2020,2009-01-01,DELSO PEREIRA SIRQUEIRA,0,3,1,2009,1/2009,200901
42854,E0001,QUITADO,21.30,26/04/2014,VENDAS,/ /,2014-04-26,JOSE CARLOS DE OLIVEIRA,0,5,4,2014,4/2014,201404
42853,E0001,QUITADO,20.00,26/04/2014,VENDAS,/ /,2014-04-26,JOSE CARLOS DE OLIVEIRA,0,5,4,2014,4/2014,201404
0,11,QUITADO,1.00,07/05/2014,VENDAS,20/05/2014,2014-05-16,NELCI RODRIGUES FERREIRA,0,4,5,2014,5/2014,201405
2569,18,QUITADO,10.62,15/05/2014,VENDAS,10/06/2014,2014-05-27,AN�ZIA LOPES NERY,0,1,5,2014,5/2014,201405
2570,18,QUITADO,22.03,17/05/2014,VENDAS,10/06/2014,2014-05-27,AN�ZIA LOPES NERY,0,1,5,2014,5/2014,201405
2571,18,QUITADO,1.96,21/05/2014,VENDAS,10/06/2014,2014-05-27,AN�ZIA LOPES NERY,0,1,5,2014,5/2014,201405
2572,18,QUITADO,5.30,21/05/2014,VENDAS,10/06/2014,2014-05-27,AN�ZIA LOPES NERY,0,1,5,2014,5/2014,201405
2573,18,QUITADO,11.23,24/05/2014,VENDAS,10/06/2014,2014-05-27,AN�ZIA LOPES NERY,0,1,5,2014,5/2014,201405
2574,18,QUITADO,8.50,27/05/2014,VENDAS,10/06/2014,2014-05-27,AN�ZIA LOPES NERY,0,1,5,2014,5/2014,201405


In [177]:
df_quitado['month_year_s'].head(len(df_quitado))

9744     200901
42854    201404
42853    201404
0        201405
2569     201405
          ...  
25841    202001
31239    202002
31238    202002
31237    202002
42814    202003
Name: month_year_s, Length: 42314, dtype: object

In [157]:
df_month_year

month_year
1/2015    17192.00
1/2016    15006.20
1/2017    14533.29
1/2018    14861.26
1/2019    12476.57
            ...   
9/2015    18484.58
9/2016    17196.08
9/2017    13773.81
9/2018    10761.09
9/2019    13438.85
Name: valor_debito, Length: 72, dtype: float64

In [150]:
'1'.

'01'